In [208]:
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import numpy as np
import psycopg2
import uuid
from datetime import datetime, date, time, timedelta

In [47]:
driver = webdriver.PhantomJS(executable_path=r'C:\phantomjs-2.1.1-windows\bin\phantomjs.exe')
#driver2 = webdriver.PhantomJS(executable_path=r'C:\phantomjs-2.1.1-windows\bin\phantomjs.exe')
driver.get("https://esports-betonline.ultraplay.net/esports/today")
#driver2.get("https://esports-betonline.ultraplay.net/esports/program/0")

sleep(5)
html = driver.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
#html2 = driver2.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
soup = BeautifulSoup(html, "html.parser")
#soup2 = BeautifulSoup(html2, "html.parser")


In [186]:
# get soup
print(soup)

<head><style type="text/css">.ng-animate.item:not(.left):not(.right){-webkit-transition:0s ease-in-out left;transition:0s ease-in-out left}</style><style type="text/css">@charset "UTF-8";[ng\:cloak],[ng-cloak],[data-ng-cloak],[x-ng-cloak],.ng-cloak,.x-ng-cloak,.ng-hide:not(.ng-hide-animate){display:none !important;}ng\:form{display:block;}</style>
<title class="ng-binding ng-scope" data-ng-bind="getTitle()" data-ng-controller="TitleController" data-ng-init="setTitle('betonline')">Today - BetOnline</title>
<meta charset="utf-8"/>
<base href="/"/>
<meta content="!" name="fragment"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="INDEX, FOLLOW" name="ROBOTS"/>
<meta class="ng-scope" content="Join betonline" data-ng-controller="TitleController" data-ng-init="setMetaDescription('Join betonline')" name="description"/>
<link href="/Content/images/fav-icon.png" rel="shortcut icon" type="image/x-icon"/>
<link href="data:;base64,iVBORw0KGgo=" rel="icon"/>


In [126]:
# connect to postgresql
conn = psycopg2.connect(dbname="BetOnline", user="Admin1", password="1407athNBC!", host = 'betonlinedb.cfitahvq3xgc.us-west-1.rds.amazonaws.com')
cur = conn.cursor()

In [212]:
inputDate = datetime.now().replace(microsecond=0).replace(second=0)
cutoffDate = inputDate + timedelta(days=-3)


oddsDF = pd.read_sql('SELECT * FROM "BetOdds" WHERE "EntryTime" < ' + str(cutoffDate), con=conn)
matchDF = pd.read_sql('SELECT * FROM "BetMatch" WHERE "Date" < ' + str(cutoffDate), con=conn)
teamDF = pd.read_sql('SELECT * FROM "BetTeam"', con=conn)
leagueDF = pd.read_sql('SELECT * FROM "BetLeague"', con=conn)
sportDF = pd.read_sql('SELECT * FROM "BetSport"', con=conn)

i = 0

sportList = soup.find_all('div',{'class': 'panel panel-default panel-sport visible-lg ng-scope'})
for sport in sportList:
    sportName = sport.find('div',{'class': 'col-xs-6 ng-binding'}).text.strip()

    # look up sport ID, otherwise add it
    #print('Inputting: '+sportName)
    SportID = sportDBCheck(sportName, sportDF, conn)
    
    leagueList = sport.find_all('div', {'class': 'panel panel-default margin-bottom-none panel-league ng-scope'})
    for league in leagueList:
        leagueName = league.find('div', {'class': 'col-xs-10 col-sm-11 text-left panel-heading match-detail-league-name'}).text.strip() 
        
        #print('Inputting: '+leagueName)
        LeagueID = leagueDBCheck(leagueName, SportID, leagueDF, conn)
        
        gameList = league.find_all('div', {'class': 'panel-match-detail ng-scope'})
        
        for game in gameList:
            teamA = game.find('div', {'class':'match-detail-teamA-row'})
            teamB = game.find('div', {'class':'match-detail-teamB-row'})
            
            # get and format date
            gameDate = game.find('div',{'class':'col-xs-2 col-sm-1 match-detail-time'}).text.strip()
            today = datetime.today()
            gameDate = datetime.strptime(date1, '%b %d %H:%M')
            gameDate=gameDate.replace(year=today.year)
            
            
            if teamA is not None:
                teamName1 = teamA.find('div', {'class': 'match-detail-team-name'}).text.strip()
                teamName2 = teamB.find('div', {'class': 'match-detail-team-name'}).text.strip()
                Odds1 = teamA.find_all('div', {'class': 'col-xs-2 col-sm-1 event-link'})
                Odds2 = teamB.find_all('div', {'class': 'col-xs-2 col-sm-1 event-link'})

                if 'suspended' not in (game.get('class')):
                    ML1 = Odds1[0].text.strip()
                    ML2 = Odds2[0].text.strip()   

                TeamID1 = teamDBCheck(teamName1, LeagueID, teamDF, conn)
                TeamID2 = teamDBCheck(teamName2, LeagueID, teamDF, conn)

                matchID = matchDBCheck(LeagueID, TeamID1, TeamID2, str(gameDate), matchDF, conn) 
                oddsID1 = oddsDBCheck(matchID, TeamID1, ML1, str(inputDate), oddsDF, conn) 
                oddsID2 = oddsDBCheck(matchID, TeamID2, ML2, str(inputDate), oddsDF, conn) 

2017-12-15 12:50:00


InterfaceError: connection already closed

In [182]:
# return league ID if it exists, otherwise create a new entry and return sportID
def oddsDBCheck (MatchID,TeamID, moneyline, timestamp, oddsDF, connection):
    moneyline = float(moneyline)

    try: 
        oddsID = oddsDF[(oddsDF['MatchID'] == MatchID)&(oddsDF['TeamID'] == TeamID)&(oddsDF['Moneyline'] == moneyline)&(oddsDF['EntryTime'] == timestamp)].iloc[0]['OddsID']
        print('found odds')
    except Exception as e:
        print("Inputting odds")        
        oddsID =  uuid.uuid4()
        queryStart = 'INSERT INTO "BetOdds" ("OddsID","MatchID","TeamID","EntryTime","Moneyline") VALUES '
        queryEnd = [oddsID, MatchID, TeamID, timestamp]
        query = queryStart + str(tuple(queryEnd))
        query = query[:-1]+", "+ str(moneyline) + ")"
        #print('match query = ' + query)
        
        try:
            cur.execute(query)
        except Exception as e: 
            print(e)
            print("error committing odds sql code")
            pass
        conn.commit()
    return oddsID    

In [181]:
# return league ID if it exists, otherwise create a new entry and return sportID
def matchDBCheck (LeagueID,TeamID1,TeamID2, timestamp, matchDF, connection):
    try: 
        matchID = matchDF[(matchDF['LeagueID'] == LeagueID)&(matchDF['TeamID1'] == TeamID1)&(matchDF['TeamID2'] == TeamID2)&(matchDF['Date'] == timestamp)].iloc[0]['MatchID']
        print('found match')
    except Exception as e:
        print("Inputting match")        
        matchID =  uuid.uuid4()
        queryStart = 'INSERT INTO "BetMatch" ("MatchID","LeagueID","TeamID1","TeamID2","Date") VALUES '
        queryEnd = [matchID, LeagueID, TeamID1, TeamID2, timestamp]
        query = queryStart + str(tuple(queryEnd))
        print('match query = ' + query)
        
        try:
            cur.execute(query)
        except Exception as e: 
            print(e)
            print("error committing match sql code")
            pass
        conn.commit()
    return matchID    

In [180]:
# return league ID if it exists, otherwise create a new entry and return sportID
def teamDBCheck (teamName, LeagueID, teamDF, connection):
   
    try: 
        teamID = teamDF[(teamDF['TeamName'] == teamName)&(teamDF['LeagueID'] == LeagueID)].iloc[0]['TeamID']
        print('found team')
    except Exception as e:
        print("Inputting team")        
        teamID =  uuid.uuid4()
        queryStart = 'INSERT INTO "BetTeam" ("TeamID","TeamName","LeagueID") VALUES '
        queryEnd = [teamID,teamName,LeagueID]
        query = queryStart + str(tuple(queryEnd))
        
        print('match query = ' + query)
        try:
            cur.execute(query)
        except Exception as e: 
            print("error committing team sql code")
            pass
        conn.commit()
    return teamID
        

In [179]:
# return league ID if it exists, otherwise create a new entry and return sportID
def leagueDBCheck (leagueName, SportID, leagueDF, connection):
    try: 
        leagueID = leagueDF[(leagueDF['LeagueName'] == leagueName)&(leagueDF['SportID'] == SportID)].iloc[0]['LeagueID']
        print('found league')
    except Exception as e:
        print("inputting league")
        leagueID =  uuid.uuid4()
        queryStart = 'INSERT INTO "BetLeague" ("LeagueID","LeagueName","SportID") VALUES '
        queryEnd = [leagueID,leagueName,SportID]
        query = queryStart + str(tuple(queryEnd))
        
        #print('league query = ' + query)
        try:
            cur.execute(query)
        except Exception as e: 
            print("error committing league sql code")
            pass
        conn.commit()
    return leagueID
        

In [175]:
# return sport ID if it exists, otherwise create a new entry and return sportID
def sportDBCheck (sportName, sportDF, connection):
    try: 
        SportID = sportDF[sportDF['SportName'] == sportName].iloc[0]['SportID']
        print('found sport')
    except:
        SportID =  uuid.uuid4() #str(uuid.uuid4()) 
        queryStart = 'INSERT INTO "BetSport" ("SportID","SportName") VALUES '
        queryEnd = [SportID,sportName]
        query = queryStart + str(tuple(queryEnd))
        
        try:
            cur.execute(query)
        except Exception as e: 
            print(e)
            print("error committing sport sql code")
            pass
        conn.commit()
    return SportID

In [ ]:
cur.close()
conn.close()